## Configuration.

Mount Google Drive onto `/content/gdrive`. You can skip this if you want to run this locally.

In [7]:
from pathlib import Path
import concurrent.futures
import numpy as np
import cv2
import imageio
from PIL import Image
import pdb
import shutil
import json

# @markdown The base directory for all captures. This can be anything if you're running this notebook on your own Jupyter runtime.
save_dir = './data/'  # @param {type: 'string'}
# @markdown The name of this capture. The working directory will be `$save_dir/$capture_name`. **Make sure you change this** when processing a new video.
capture_name = 'kubric_single_car_rand_v2'  # @param {type: 'string'}
# The root directory for this capture.
root_dir = Path(save_dir, capture_name)
# Where to save RGB images.
rgb_raw_dir = root_dir / 'rgb-raw'
# Where to save mask images.
mask_dir = root_dir / 'mask'
mask_raw_dir = root_dir / 'mask-raw'
# Where to save the COLMAP outputs.
colmap_dir = root_dir
colmap_db_path = colmap_dir / 'database.db'
colmap_out_path = colmap_dir / 'sparse'

freeze_test_dir = root_dir / 'freeze-test'
if freeze_test_dir.exists():
    rgb_raw_freeze_test_dir = freeze_test_dir / 'rgb-raw'
    cam_freeze_test_dir = freeze_test_dir / 'camera-gt'

colmap_out_path.mkdir(exist_ok=True, parents=True)
rgb_raw_dir.mkdir(exist_ok=True, parents=True)
mask_dir.mkdir(exist_ok=True, parents=True)

print(f"""Directories configured:
  root_dir = {root_dir}
  rgb_raw_dir = {rgb_raw_dir}
  colmap_dir = {colmap_dir}
  mask_raw_dir = {mask_raw_dir}
  mask_dir = {mask_dir}
""")

tmp_rgb_raw_dir = rgb_raw_dir

rgb_list = list(sorted(rgb_raw_dir.glob( '*.png')))
image_size = imageio.imread(rgb_list[0]).shape[:2]

target_image_dir = root_dir / 'images'
target_image_dir.mkdir(exist_ok=True)
for rgb in rgb_list:
    shutil.copy(str(rgb), str(target_image_dir))
    
if freeze_test_dir.exists():
    freeze_rgb_list = list(sorted(rgb_raw_freeze_test_dir.glob( '*.png')))
    for rgb in freeze_rgb_list:
        shutil.copy(str(rgb), str(target_image_dir / f'test_{rgb.stem}.png'))


# @title Save dataset split to `dataset_nsff.json`.
target_rgb_list = list(sorted(target_image_dir.glob( '*.png')))

with open(root_dir / 'dataset.json','r') as f:
    dataset_info = json.load(f)

all_ids = list(map(lambda x: x.stem, target_rgb_list))
val_ids = dataset_info['val_ids']
test_ids = [id for id in all_ids if 'test' in id]
train_ids = sorted(set(all_ids) - set(val_ids) - set(test_ids))

print(f'Num of train: {len(train_ids)}')
print(f'Num of val: {len(val_ids)}')

dataset_json = {
    'count': len(all_ids),
    'num_exemplars': len(train_ids),
    'ids': all_ids,
    'train_ids': train_ids,
    'val_ids': val_ids,
    'test_ids': test_ids
}

dataset_json_path = root_dir / 'dataset_nsff.json'
with dataset_json_path.open('w') as f:
    json.dump(dataset_json, f, indent=2)

# add freeze tests metadata
if freeze_test_dir.exists():
    with open(root_dir / 'metadata.json','r') as f:
        metadata_info = json.load(f)

    with open(freeze_test_dir / 'metadata.json','r') as f:
        freeze_metadata_info = json.load(f)

    for rgb in freeze_rgb_list:
        rgb_name = rgb.stem
        metadata_info[f'test_{rgb_name}'] = freeze_metadata_info[rgb_name]
    
    with (root_dir / 'metadata_nsff.json').open('w') as f:
        json.dump(metadata_info, f, indent=2)

Directories configured:
  root_dir = data/kubric_single_car_rand_v2
  rgb_raw_dir = data/kubric_single_car_rand_v2/rgb-raw
  colmap_dir = data/kubric_single_car_rand_v2
  mask_raw_dir = data/kubric_single_car_rand_v2/mask-raw
  mask_dir = data/kubric_single_car_rand_v2/mask

Num of train: 200
Num of val: 0


## Dataset Processing.

### Load Video.

In this step we upload a video file and flatten it into PNG files using ffmpeg.

### Do the same for mask

### Camera registration with COLMAP.

In [2]:
# @title Extract features.
# @markdown Computes SIFT features and saves them to the COLMAP DB.
share_intrinsics = True  # @param {type: 'boolean'}
assume_upright_cameras = True  # @param {type: 'boolean'}

# @markdown This sets the scale at which we will run COLMAP. A scale of 1 will be more accurate but will be slow.
colmap_rgb_dir = target_image_dir

# @markdown Check this if you want to re-process SfM.
overwrite = True  # @param {type: 'boolean'}

if overwrite and colmap_db_path.exists():
  colmap_db_path.unlink()

!colmap feature_extractor \
--SiftExtraction.use_gpu 0 \
--SiftExtraction.upright {int(assume_upright_cameras)} \
--ImageReader.camera_model OPENCV \
--ImageReader.single_camera {int(share_intrinsics)} \
--database_path "{str(colmap_db_path)}" \
--image_path "{str(colmap_rgb_dir)}"



Feature extraction

Processed file [1/300]
  Name:            rgba_00001.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1498
Processed file [2/300]
  Name:            rgba_00002.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1555
Processed file [3/300]
  Name:            rgba_00005.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1608
Processed file [4/300]
  Name:            rgba_00004.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1594
Processed file [5/300]
  Name:            rgba_00003.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1573
Processed file [6/300]
  Name:            rgba_00000.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:  

Processed file [49/300]
  Name:            rgba_00048.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1724
Processed file [50/300]
  Name:            rgba_00051.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1741
Processed file [51/300]
  Name:            rgba_00049.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1735
Processed file [52/300]
  Name:            rgba_00052.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1768
Processed file [53/300]
  Name:            rgba_00053.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1778
Processed file [54/300]
  Name:            rgba_00050.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Fe

Processed file [98/300]
  Name:            rgba_00097.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1660
Processed file [99/300]
  Name:            rgba_00098.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1620
Processed file [100/300]
  Name:            rgba_00099.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1663
Processed file [101/300]
  Name:            rgba_00100.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1631
Processed file [102/300]
  Name:            rgba_00101.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1641
Processed file [103/300]
  Name:            rgba_00102.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px


Processed file [147/300]
  Name:            rgba_00146.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1619
Processed file [148/300]
  Name:            rgba_00147.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1627
Processed file [149/300]
  Name:            rgba_00149.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1613
Processed file [150/300]
  Name:            rgba_00148.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1615
Processed file [151/300]
  Name:            rgba_00150.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1623
Processed file [152/300]
  Name:            rgba_00151.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40p

Processed file [196/300]
  Name:            rgba_00193.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1645
Processed file [197/300]
  Name:            rgba_00192.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1649
Processed file [198/300]
  Name:            rgba_00198.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1628
Processed file [199/300]
  Name:            rgba_00197.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1661
Processed file [200/300]
  Name:            rgba_00199.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1648
Processed file [201/300]
  Name:            test_rgba_00000.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    61

Processed file [244/300]
  Name:            test_rgba_00045.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1562
Processed file [245/300]
  Name:            test_rgba_00043.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1552
Processed file [246/300]
  Name:            test_rgba_00046.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1556
Processed file [247/300]
  Name:            test_rgba_00047.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1542
Processed file [248/300]
  Name:            test_rgba_00044.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1547
Processed file [249/300]
  Name:            test_rgba_00049.png
  Dimensions:      512 x 512
  Camera:          #1 - OPE

Processed file [293/300]
  Name:            test_rgba_00093.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1722
Processed file [294/300]
  Name:            test_rgba_00092.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1722
Processed file [295/300]
  Name:            test_rgba_00094.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1763
Processed file [296/300]
  Name:            test_rgba_00095.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1729
Processed file [297/300]
  Name:            test_rgba_00096.png
  Dimensions:      512 x 512
  Camera:          #1 - OPENCV
  Focal Length:    614.40px
  Features:        1713
Processed file [298/300]
  Name:            test_rgba_00097.png
  Dimensions:      512 x 512
  Camera:          #1 - OPE

In [3]:
# @title Match features.
# @markdown Match the SIFT features between images. Use `exhaustive` if you only have a few images and use `vocab_tree` if you have a lot of images.

match_method = 'exhaustive'  # @param ["exhaustive", "vocab_tree"]

if match_method == 'exhaustive':
  !colmap exhaustive_matcher \
    --SiftMatching.use_gpu 0 \
    --database_path "{str(colmap_db_path)}"
else:
  # Use this if you have lots of frames.
  !wget https://demuc.de/colmap/vocab_tree_flickr100K_words32K.bin
  !colmap vocab_tree_matcher \
    --VocabTreeMatching.vocab_tree_path vocab_tree_flickr100K_words32K.bin \
    --SiftMatching.use_gpu 0 \
    --database_path "{str(colmap_db_path)}"


Exhaustive feature matching

Matching block [1/6, 1/6] in 66.926s
Matching block [1/6, 2/6] in 68.545s
Matching block [1/6, 3/6] in 64.444s
Matching block [1/6, 4/6] in 64.633s
Matching block [1/6, 5/6] in 69.493s
Matching block [1/6, 6/6] in 63.316s
Matching block [2/6, 1/6] in 65.123s
Matching block [2/6, 2/6] in 62.561s
Matching block [2/6, 3/6] in 58.234s
Matching block [2/6, 4/6] in 60.682s
Matching block [2/6, 5/6] in 64.732s
Matching block [2/6, 6/6] in 61.085s
Matching block [3/6, 1/6] in 64.617s
Matching block [3/6, 2/6] in 64.915s
Matching block [3/6, 3/6] in 58.095s
Matching block [3/6, 4/6] in 60.171s
Matching block [3/6, 5/6] in 63.928s
Matching block [3/6, 6/6] in 58.885s
Matching block [4/6, 1/6] in 64.247s
Matching block [4/6, 2/6] in 64.796s
Matching block [4/6, 3/6] in 61.967s
Matching block [4/6, 4/6] in 60.417s
Matching block [4/6, 5/6] in 65.708s
Matching block [4/6, 6/6] in 60.788s
Matching block [5/6, 1/6] in 67.273s
Matching block [5/6, 2/6] in 69.418s
Matching

In [4]:
# @title Reconstruction.
# @markdown Run structure-from-motion to compute camera parameters.

refine_principal_point = True  #@param {type:"boolean"}
min_num_matches = 32 # @param {type: 'number'}
filter_max_reproj_error = 2  # @param {type: 'number'}
tri_complete_max_reproj_error = 2  # @param {type: 'number'}

!colmap mapper \
  --Mapper.ba_refine_principal_point {int(refine_principal_point)} \
  --Mapper.filter_max_reproj_error $filter_max_reproj_error \
  --Mapper.tri_complete_max_reproj_error $tri_complete_max_reproj_error \
  --Mapper.min_num_matches $min_num_matches \
  --database_path "{str(colmap_db_path)}" \
  --image_path "{str(colmap_rgb_dir)}" \
  --output_path "{str(colmap_out_path)}" # renamed to output path


Loading database

Loading cameras... 1 in 0.000s
Loading matches... 44831 in 0.524s
Loading images... 300 in 0.042s (connected 300)
Building correspondence graph... in 8.120s (ignored 497)

Elapsed time: 0.145 [minutes]


Initializing with image pair #65 and #262


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  4.212074e+03    0.00e+00    3.01e+04   0.00e+00   0.00e+00  1.00e+04        0    1.74e-03    7.47e-03
   1  3.190778e+04   -2.77e+04    0.00e+00   5.53e+01  -2.94e+01  5.00e+03        1    4.09e-03    1.16e-02
   2  1.283517e+04   -8.62e+03    0.00e+00   2.72e+01  -9.80e+00  1.25e+03        1    2.61e-03    1.43e-02
   3  4.229860e+03   -1.78e+01    0.00e+00   7.17e+00  -2.27e-02  1.56e+02        1    2.38e-03    1.67e-02
   4  3.575342e+03    6.37e+02    1.72e+04   1.62e+00   9.19e-01  3.79e+02        1    3.00e-03    1.98e-02
   5  3.556194e+03    1.91e+01    9.88e+03   1.69e+00   9.0

  81  1.674779e+02    1.63e+01    4.12e+04   2.81e+01   8.05e-01  2.11e+04        1    4.11e-03    2.62e-01
  82  1.555598e+02    1.19e+01    3.68e+04   1.71e+01   7.90e-01  2.63e+04        1    3.12e-03    2.65e-01
  83  1.467471e+02    8.81e+00    3.24e+04   5.59e+00   7.61e-01  3.06e+04        1    3.17e-03    2.69e-01
  84  1.400623e+02    6.68e+00    2.95e+04   1.49e+01   7.45e-01  3.47e+04        1    3.00e-03    2.72e-01
  85  1.350197e+02    5.04e+00    2.92e+04   2.63e+01   7.22e-01  3.81e+04        1    3.01e-03    2.75e-01
  86  1.310509e+02    3.97e+00    2.92e+04   3.43e+01   7.05e-01  4.09e+04        1    3.05e-03    2.78e-01
  87  1.278342e+02    3.22e+00    3.02e+04   3.94e+01   6.81e-01  4.29e+04        1    3.12e-03    2.81e-01
  88  1.250584e+02    2.78e+00    3.56e+04   4.20e+01   6.67e-01  4.45e+04        1    3.03e-03    2.84e-01
  89  1.225742e+02    2.48e+00    3.93e+04   4.32e+01   6.54e-01  4.59e+04        1    3.02e-03    2.87e-01
  90  1.202604e+02    2.31e+

  => Could not register, trying another image.

Registering image #48 (3)

  => Image sees 484 / 1642 points
  => Could not register, trying another image.

Registering image #69 (3)

  => Image sees 478 / 1673 points
  => Could not register, trying another image.

Registering image #196 (3)

  => Image sees 468 / 1629 points
  => Could not register, trying another image.

Registering image #190 (3)

  => Image sees 468 / 1665 points
  => Could not register, trying another image.

Retriangulation

  => Merged observations: 0
  => Completed observations: 0
  => Retriangulated observations: 0

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.945225e+01    0.00e+00    5.05e+04   0.00e+00   0.00e+00  1.00e+04        0    1.19e-03    5.63e-03
   1  3.397748e+01    5.47e+00    5.03e+03   1.82e+01   9.92e-01  3.00e+04        1    3.89e-03    9.59e-03
   2  3.125381e+01    2.72e+00    2.37e+04   3.86e+

  68  1.420742e+01    1.52e-02    3.66e+03   6.72e+00   7.14e-01  1.99e+06        1    5.57e-03    2.32e-01
  69  1.419125e+01    1.62e-02    3.70e+03   6.96e+00   7.16e-01  2.17e+06        1    3.39e-03    2.35e-01
  70  1.417399e+01    1.73e-02    3.70e+03   7.22e+00   7.21e-01  2.37e+06        1    3.30e-03    2.39e-01
  71  1.415547e+01    1.85e-02    3.65e+03   7.52e+00   7.26e-01  2.62e+06        1    3.17e-03    2.42e-01
  72  1.413549e+01    2.00e-02    3.52e+03   7.86e+00   7.31e-01  2.90e+06        1    3.01e-03    2.45e-01
  73  1.411380e+01    2.17e-02    3.25e+03   8.26e+00   7.35e-01  3.24e+06        1    2.94e-03    2.48e-01
  74  1.409025e+01    2.36e-02    2.75e+03   8.70e+00   7.32e-01  3.60e+06        1    2.96e-03    2.51e-01
  75  1.406489e+01    2.54e-02    1.87e+03   9.12e+00   7.19e-01  3.93e+06        1    3.17e-03    2.54e-01
  76  1.403831e+01    2.66e-02    5.57e+02   9.37e+00   6.88e-01  4.15e+06        1    3.20e-03    2.57e-01
  77  1.401114e+01    2.72e-


Initializing with image pair #65 and #259


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.502045e+02    0.00e+00    7.51e+03   0.00e+00   0.00e+00  1.00e+04        0    1.17e-03    6.62e-03
   1  2.299603e+02   -7.98e+01    0.00e+00   3.04e+01  -1.62e+00  5.00e+03        1    3.77e-03    1.04e-02
   2  1.267145e+02    2.35e+01    3.67e+04   1.76e+01   5.72e-01  5.02e+03        1    3.46e-03    1.39e-02
   3  1.056002e+02    2.11e+01    2.47e+04   1.45e+01   8.50e-01  7.64e+03        1    3.79e-03    1.77e-02
   4  1.051114e+02    4.89e-01    4.20e+04   1.87e+01   3.98e-02  4.29e+03        1    3.71e-03    2.15e-02
   5  9.031067e+01    1.48e+01    1.00e+04   9.57e+00   9.58e-01  1.29e+04        1    3.60e-03    2.51e-02
   6  1.030131e+02   -1.27e+01    0.00e+00   2.15e+01  -1.52e+00  6.44e+03        1    2.93e-03    2.80e-02
   7  8.716158e+01    3.15e+00    1.67e+04   1.22e+01   6.17e-01 

...

   1  3.500099e+04    1.28e+03    7.47e+01   9.74e+00   1.00e+00  3.00e+04        1    1.20e+01    1.43e+01
   2  3.500047e+04    5.12e-01    1.56e+01   1.35e+00   1.00e+00  9.00e+04        1    2.85e+00    1.72e+01
   3  3.500047e+04    8.97e-03    5.77e+01   2.36e-01   1.00e+00  2.70e+05        1    2.82e+00    2.00e+01
   4  3.500046e+04    6.61e-03    1.91e+02   3.71e-01   9.99e-01  8.10e+05        1    2.85e+00    2.29e+01
   5  3.500045e+04    4.81e-03    4.56e+02   4.26e-01   9.98e-01  2.43e+06        1    2.82e+00    2.57e+01
   6  3.500045e+04    1.88e-03    3.21e+02   3.00e-01   9.99e-01  7.29e+06        1    2.82e+00    2.85e+01
   7  3.500045e+04    2.01e-04    4.50e+01   1.08e-01   1.01e+00  2.19e+07        1    2.82e+00    3.13e+01
   8  3.500045e+04    3.99e-06    8.98e-01   1.54e-02   1.02e+00  6.56e+07        1    2.83e+00    3.42e+01


Bundle adjustment report
------------------------
    Residuals : 846210
   Parameters : 25927
   Iterations : 9
         Time : 34.22

  => Merged observations: 851
  => Completed observations: 586
  => Filtered observations: 167
  => Changed observations: 0.163908

Registering image #60 (287)

  => Image sees 1578 / 1644 points

Pose refinement report
----------------------
    Residuals : 3504
   Parameters : 6
   Iterations : 4
         Time : 0.0171712 [s]
 Initial cost : 0.367102 [px]
   Final cost : 0.347099 [px]
  Termination : Convergence

  => Continued observations: 1570
  => Added observations: 3179

Bundle adjustment report
------------------------
    Residuals : 25584
   Parameters : 580
   Iterations : 14
         Time : 0.205671 [s]
 Initial cost : 0.328425 [px]
   Final cost : 0.319935 [px]
  Termination : Convergence

  => Merged observations: 14072
  => Completed observations: 641
  => Filtered observations: 3133
  => Changed observations: 1.395091

Bundle adjustment report
------------------------
    Residuals : 19358
   Parameters : 181
   Iterations : 3
         Time : 0.0412693 [s]
 Initial cos

  => Merged observations: 12974
  => Completed observations: 396
  => Filtered observations: 2276
  => Changed observations: 1.336351

Bundle adjustment report
------------------------
    Residuals : 19324
   Parameters : 229
   Iterations : 3
         Time : 0.0397128 [s]
 Initial cost : 0.234921 [px]
   Final cost : 0.230596 [px]
  Termination : Convergence

  => Merged observations: 585
  => Completed observations: 279
  => Filtered observations: 87
  => Changed observations: 0.098427

Registering image #57 (294)

  => Image sees 1565 / 1625 points

Pose refinement report
----------------------
    Residuals : 3534
   Parameters : 6
   Iterations : 4
         Time : 0.0160036 [s]
 Initial cost : 0.372794 [px]
   Final cost : 0.359087 [px]
  Termination : Convergence

  => Continued observations: 1554
  => Added observations: 1974

Bundle adjustment report
------------------------
    Residuals : 23346
   Parameters : 553
   Iterations : 15
         Time : 0.203582 [s]
 Initial cost

  => Added observations: 1744

Bundle adjustment report
------------------------
    Residuals : 24258
   Parameters : 1033
   Iterations : 19
         Time : 0.281084 [s]
 Initial cost : 0.336842 [px]
   Final cost : 0.33093 [px]
  Termination : Convergence

  => Merged observations: 15140
  => Completed observations: 312
  => Filtered observations: 2266
  => Changed observations: 1.460796

Bundle adjustment report
------------------------
    Residuals : 20890
   Parameters : 649
   Iterations : 3
         Time : 0.0445847 [s]
 Initial cost : 0.280588 [px]
   Final cost : 0.27571 [px]
  Termination : Convergence

  => Merged observations: 653
  => Completed observations: 173
  => Filtered observations: 140
  => Changed observations: 0.092484

Retriangulation

  => Merged observations: 825
  => Completed observations: 17989
  => Retriangulated observations: 18

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  to

  => Merged observations: 458
  => Completed observations: 1009
  => Filtered observations: 961
  => Changed observations: 0.005422

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.638244e+04    0.00e+00    2.16e+04   0.00e+00   0.00e+00  1.00e+04        0    3.42e-01    2.48e+00

CHOLMOD version 3.0.14, Oct 22, 2019: Symbolic Analysis: status: OK
  Architecture: Linux
    sizeof(int):      4
    sizeof(SuiteSparse_long):  8
    sizeof(void *):   8
    sizeof(double):   8
    sizeof(Int):      4 (CHOLMOD's basic integer)
    sizeof(BLAS_INT): 4 (integer used in the BLAS)
  Results from most recent analysis:
    Cholesky flop count: 1.9489e+09
    Nonzeros in L:       1.6227e+06
  memory blocks in use:          11
  memory in use (MB):           0.1
  peak memory usage (MB):       6.3
  maxrank:    update/downdate rank:   8
  supernodal control: 1 40 (supernodal if flops/lnz >= 40)
  nmethods: 

In [5]:
# @title Verify that SfM worked.

if not colmap_db_path.exists():
  raise RuntimeError(f'The COLMAP DB does not exist, did you run the reconstruction?')
elif not (colmap_dir / 'sparse/0/cameras.bin').exists():
  raise RuntimeError("""
SfM seems to have failed. Try some of the following options:
 - Increase the FPS when flattenting to images. There should be at least 50-ish images.
 - Decrease `min_num_matches`.
 - If you images aren't upright, uncheck `assume_upright_cameras`.
""")
else:
  print("Everything looks good!")

Everything looks good!
